In [1]:
%matplotlib inline
import os
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from poison_evaluation.models import ResNet18
from tqdm import tqdm

In [2]:
# cifar_path = '/vulcanscratch/psando/cifar-10'
# transform_test = transforms.Compose([
#         transforms.ToTensor(),
#         transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
#     ])

# testset = torchvision.datasets.CIFAR10(
#         root=cifar_path, train=True, download=False, transform=transform_test)
# testloader = torch.utils.data.DataLoader(
#     testset, batch_size=100, shuffle=False, num_workers=1)

In [3]:
# import village 
# cifar_ckpt_path = '/vulcanscratch/psando/cifar_model_ckpts'
# model_name = 'ResNet18'
# cifar_ckpt = os.path.join(cifar_ckpt_path, model_name + '.pt')
# ckpt = torch.load(cifar_ckpt)

# model = village.clients.models.resnet_picker('ResNet18', 'CIFAR10')
# model.load_state_dict(ckpt['model'])
# model = model.cuda()
# model.eval()
# pass

In [4]:
# Save representation layer features

# features = []
# def log_feature(module, input, output):
#     batch_size = output.size(0)
#     features.append(output.reshape(batch_size, -1))
#     return None

# handle = model.avgpool.register_forward_hook(log_feature)

# with torch.no_grad():
#     for x, y in tqdm(testloader):
#         x, y = x.cuda(), y.cuda()
#         o = model(x)
        
# handle.remove()

# features = torch.cat(features, dim=0) # shape (50000, 512)
# print(features.shape)
# torch.save(features, 'trainset_features.pt')

# Load features
features = torch.load('trainset_features.pt') # shape (50000, 512)

In [5]:
num_principal_dirs = 6
pca_iters = 100
usv = torch.pca_lowrank(features, q=num_principal_dirs, center=True, niter=pca_iters)

In [6]:
# Check covariance of num_principal_dirs 
S = usv[1]
cov = S ** 2 / (features.size(0) - 1)
print(cov)

tensor([4.6434, 3.4170, 3.0933, 2.5911, 2.5174, 2.2172], device='cuda:0')


In [7]:
# Project onto first two principal components
V = usv[2] # shape (512, 6)
proj_features = torch.matmul(features, V[:, :2]) # shape (50000, 2)

In [8]:
# Rotate every projected feature
import math
a = math.pi / 4
rot = torch.tensor([[math.cos(a), -math.sin(a)], [math.sin(a), math.cos(a)]]).cuda()
rot_proj_features = torch.matmul(proj_features, rot) # shape (50000, 2)

In [9]:
# Sanity check: dot product between normalized vectors should equal cosine of rotation angle
u = proj_features[0] / torch.norm(proj_features[0], p=2)
v = rot_proj_features[0] / torch.norm(rot_proj_features[0], p=2)
print(torch.dot(u, v))
print(math.cos(a))

tensor(0.7071, device='cuda:0')
0.7071067811865476


In [10]:
# Unproject back rotated features by adding back non-principal components
rot_proj_features = torch.matmul(rot_proj_features, V[:, :2].T) # shape (50000, 512)
rot_features = rot_proj_features + (features - torch.matmul(proj_features, V[:, :2].T)) # shape (50000, 512)

In [11]:
# Sanity check: dot product between normalized vectors should equal cosine of rotation angle
u = features[0] / torch.norm(features[0], p=2)
v = rot_features[0] / torch.norm(rot_features[0], p=2)
print(torch.dot(u, v))
print(math.cos(a))

tensor(0.9600, device='cuda:0')
0.7071067811865476
